In [1]:
import py2neo
from py2neo import Graph,NodeMatcher
import neo4j
from neo4j import GraphDatabase
from py2neo.data import Node, Relationship

In [4]:
g = Graph("bolt://localhost:7687",auth=("neo4j", "test")) 
matcher = NodeMatcher(g)

In [5]:
driver = GraphDatabase.driver("neo4j://localhost:7687",auth=("neo4j", "test"))

In [6]:
import pandas as pd
df = pd.read_csv('/Users/jasminekaur/Desktop/basf/Code/data/Boston_Crime_Dataset.csv',encoding="latin_1",dtype='unicode' )  

In [7]:
df.dtypes

INCIDENT_NUMBER        object
OFFENSE_CODE           object
OFFENSE_CODE_GROUP     object
OFFENSE_DESCRIPTION    object
DISTRICT               object
REPORTING_AREA         object
OCCURRED_ON_DATE       object
YEAR                   object
MONTH                  object
DAY_OF_WEEK            object
HOUR                   object
UCR_PART               object
STREET                 object
Lat                    object
Long                   object
Location               object
dtype: object

In [8]:
df2 = df.head(50)

In [9]:
df = pd.read_csv('/Users/jasminekaur/Desktop/basf/Code/data/soc_pokec_profiles.txt',sep= '\t',header=None ) 
df_n = pd.read_csv('/Users/jasminekaur/Desktop/basf/Code/data/soc-pokec-relationships.txt',sep= '\t',header=None )

In [11]:
a = Node("Person", name="Mark", age=33)
b = Node("Person", name="Rachel", age=44)
KNOWS = Relationship.type("KNOWS")
g.merge(KNOWS(a, b), "Person", "name")

In [12]:
c = Node("Company", name="BASf")
c.__primarylabel__ = "Company"
c.__primarykey__ = "name"
WORKS_FOR = Relationship.type("WORKS_FOR")
g.merge(WORKS_FOR(a, c) | WORKS_FOR(b, c))

In [13]:
g.nodes.match("Person", name="Alice").first()

Node('Person', age=33, name='Alice')

In [14]:
len(g.nodes)

39

In [15]:
len(g.nodes.match("Person", age=33))

4

In [16]:
tx = g.begin()
a = Node("Person", name="Ross")
tx.create(a)
b = Node("Person", name="Joey")
ab = Relationship(a, "KNOWS", b)
tx.create(ab)
tx.commit()
g.exists(ab)

True

In [17]:
while cursor.forward():
    print(cursor.current["name"])

NameError: name 'cursor' is not defined

In [19]:
g.run("MATCH (a:Person) RETURN a.name, a.born ").data()

[{'a.name': 'Alice', 'a.born': None},
 {'a.name': 'Bob', 'a.born': None},
 {'a.name': 'Alice', 'a.born': None},
 {'a.name': 'Bob', 'a.born': None},
 {'a.name': 'Alice', 'a.born': None},
 {'a.name': 'Mike', 'a.born': None},
 {'a.name': 'June', 'a.born': None},
 {'a.name': 'Jane', 'a.born': None},
 {'a.name': 'Fred', 'a.born': None},
 {'a.name': 'Alex', 'a.born': None},
 {'a.name': 'Thom', 'a.born': None},
 {'a.name': 'Codu', 'a.born': None},
 {'a.name': 'Mike', 'a.born': None},
 {'a.name': 'June', 'a.born': None},
 {'a.name': 'Jane', 'a.born': None},
 {'a.name': 'Fred', 'a.born': None},
 {'a.name': 'Alex', 'a.born': None},
 {'a.name': 'Thom', 'a.born': None},
 {'a.name': 'Codu', 'a.born': None},
 {'a.name': 'Mark', 'a.born': None},
 {'a.name': 'Rachel', 'a.born': None},
 {'a.name': 'Ross', 'a.born': None},
 {'a.name': 'Joey', 'a.born': None}]

In [20]:
g.run("CREATE (a:Person) SET a.name = 'Alice'").stats()

constraints_added: 0
constraints_removed: 0
contained_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 1
labels_removed: 0
nodes_created: 1
nodes_deleted: 0
properties_set: 1
relationships_created: 0
relationships_deleted: 0

In [21]:
list(matcher.match("Person").where("_.name =~ 'A.*'").order_by("_.name").limit(3))

[Node('Person', age=26, fdg=4.33, id=5, name='Alex'),
 Node('Person', age=26, fdg=4.33, id=5, name='Alex'),
 Node('Person', name='Alice')]

In [22]:
nodedata = [{'subid': '1', 'age': 75, 'fdg': 1.78, 'name': 'Fred'},
            {'subid': '2', 'age': 33, 'fdg': 1.56, 'name': 'May'},
            {'subid': '3', 'age': 32, 'fdg': 1.11, 'name': 'Janice'},
            {'subid': '4', 'age': 77, 'fdg': 1.02, 'name': 'Alen'},
            {'subid': '5', 'age': 26, 'fdg': 4.33, 'name': 'tom'},
            {'subid': '6', 'age': 54, 'fdg': 2.11, 'name': 'carl'},
            {'subid': '7', 'age': 24, 'fdg': 5.22, 'name': 'casper'}]
nodes = pd.DataFrame(nodedata)

In [23]:

edgedata = [{'source': '1', 'dest': '2', 'weight': 1, 'rating': 2},
            {'source': '1', 'dest': '3', 'weight': 1, 'rating': 1},
            {'source': '1', 'dest': '5', 'weight': 1, 'rating': 6},
            {'source': '1', 'dest': '6', 'weight': 1, 'rating': 8},
            {'source': '6', 'dest': '7', 'weight': 1, 'rating': 3},
            {'source': '5', 'dest': '3', 'weight': 1, 'rating': 4},
            {'source': '4', 'dest': '3', 'weight': 1, 'rating': 9},
            {'source': '2', 'dest': '4', 'weight': 1, 'rating': 2}]
edges = pd.DataFrame(edgedata)

In [24]:
#cypher params
nodename = "Person"
edgename = "RATED"
source_name = "Person"
dest_name = "Person"

In [25]:
#NODE BY NODE IMPORT
for index, row in nodes.iterrows():
    node = Node(nodename, id=int(row['subid']), age = row['age'], fdg = row['fdg'], name = row['name'])
    g.create(node)

In [26]:
for index, row in edges.iterrows():
    source_node = matcher.match(nodename, property_key='id', property_value=int(row['source']))
    dest_node = matcher.match(nodename, property_key='id', property_value = int(row['dest']))
    relation = Relationship(source_node, edgename, dest_node, rating = int(row['rating']))
    g.create(relation)

In [27]:
g.run("MATCH (a:Person) RETURN a.name, a.age ").data()

[{'a.name': 'Alice', 'a.age': 33},
 {'a.name': 'Bob', 'a.age': 44},
 {'a.name': 'Alice', 'a.age': None},
 {'a.name': 'Bob', 'a.age': None},
 {'a.name': 'Alice', 'a.age': None},
 {'a.name': 'Mike', 'a.age': 75},
 {'a.name': 'June', 'a.age': 33},
 {'a.name': 'Jane', 'a.age': 32},
 {'a.name': 'Fred', 'a.age': 77},
 {'a.name': 'Alex', 'a.age': 26},
 {'a.name': 'Thom', 'a.age': 54},
 {'a.name': 'Codu', 'a.age': 24},
 {'a.name': 'Mike', 'a.age': 75},
 {'a.name': 'June', 'a.age': 33},
 {'a.name': 'Jane', 'a.age': 32},
 {'a.name': 'Fred', 'a.age': 77},
 {'a.name': 'Alex', 'a.age': 26},
 {'a.name': 'Thom', 'a.age': 54},
 {'a.name': 'Codu', 'a.age': 24},
 {'a.name': 'Mark', 'a.age': 33},
 {'a.name': 'Rachel', 'a.age': 44},
 {'a.name': 'Ross', 'a.age': None},
 {'a.name': 'Joey', 'a.age': None},
 {'a.name': 'Alice', 'a.age': None},
 {'a.name': 'Fred', 'a.age': 75},
 {'a.name': 'May', 'a.age': 33},
 {'a.name': 'Janice', 'a.age': 32},
 {'a.name': 'Alen', 'a.age': 77},
 {'a.name': 'tom', 'a.age': 26}